In [1]:
from modules.utils.database_utils import get_db_connection

conn = get_db_connection()

In [8]:
import sys
import os

os.chdir("..")
from modules.data.download import download_data
download_data("2025-06-01", "2025-08-01")

Found 1617 debates.
Found 0 left to scrape


In [ ]:
import sys
import os
import logging
logging.getLogger('httpx').setLevel(logging.WARNING)

from modules.points.generate import generate_points
generate_points(filters={"start_date": "2025-06-01", "end_date": "2025-08-01"})

Analysed: 450 Unanalysed: 336
Skipping contribution 44671162 - no member ID
Skipping contribution 3312785 - no member ID
Skipping contribution 44713761 - no member ID
Skipping contribution 3316871 - no member ID
Skipping contribution 44634618 - no member ID
Skipping contribution 3309534 - no member ID


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Generating embedding for point: wants to know the steps the Department of Health is taking to tackle health inequalities across the country
Generating embedding for point: wants to know the estimated number of additional jobs created in clean power industries in the north-east since July 2024
Generating embedding for point: wants the Department to ensure areas with higher levels of deprivation, such as Newcastle-under-Lyme, receive adequate funding


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Generating embedding for point: residents of Pankhurst House and neighbouring blocks in Ealing Central and Acton are concerned about poor service from Clarion Housing
Generating embedding for point: notes the Parliamentary and Health Service Ombudsman's report on Women's State Pension Age, which found maladministration in the Department for Work and Pensions' communication about the Pensions Act 1995
Generating embedding for point: believes further education institutions are vital to the British economy


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Generating embedding for point: Opposes potential review to scale back benefits of draft beer duty rate


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Generating embedding for point: believes further education colleges are vital for vocational training, apprenticeships, and addressing skills gaps
Generating embedding for point: Banking services are crucial to the future of the Post Office and the government wants to improve collaboration between the Post Office and banks
Generating embedding for point: The UK should join in any secondary sanctions policy directed against countries importing Russian oil and gas
Generating embedding for point: thinks the government is prioritising cost over fairness for women affected by the change in state pension age


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [5]:
import numpy as np
from psycopg2.extras import execute_values

def to_fp16_bytes(x32, dims=256):
    return np.asarray(x32, dtype=np.float32)[:dims].astype(np.float16).tobytes()

def backfill_emb256_f16(conn, dims=256, batch=5000):
    with conn, conn.cursor(name="full_vectors") as rcur:
        rcur.itersize = batch
        rcur.execute("""
            SELECT point_id, (point_embedding::real[]) AS emb
            FROM point
            WHERE point_embedding IS NOT NULL AND emb256_f16 IS NULL
        """)
        buf = []
        with conn.cursor() as wcur:
            for rows in iter(lambda: rcur.fetchmany(batch), []):
                for pid, emb in rows:
                    buf.append((pid, to_fp16_bytes(emb, dims)))
                execute_values(
                    wcur,
                    """
                    UPDATE point AS p
                    SET emb256_f16 = data.emb
                    FROM (VALUES %s) AS data(point_id, emb)
                    WHERE p.point_id = data.point_id
                    """,
                    buf,
                    template="(%s,%s)"
                )
                buf.clear()


In [6]:
backfill_emb256_f16(get_db_connection())

In [ ]:
import psycopg2
from typing import Dict
import base64, json

def get_db_connection():
    return psycopg2.connect(
        host="localhost",
        database="parliament",
        user="parliament_user",
        password="parliament_pass",
        port=5432
    )

def create_cluster_jobs_and_clusters_tables():
    conn = get_db_connection()
    cur = conn.cursor()
    # Create cluster_jobs table
    cur.execute("""
    CREATE TABLE IF NOT EXISTS cluster_jobs (
      job_id        BIGSERIAL PRIMARY KEY,
      status        TEXT NOT NULL CHECK (status IN ('queued','running','complete','failed','canceled')),
      root_cluster_id INTEGER REFERENCES clusters(cluster_id) ON DELETE SET NULL,
      params        JSONB NOT NULL,
      created_at    TIMESTAMPTZ NOT NULL DEFAULT now(),
      started_at    TIMESTAMPTZ,
      finished_at   TIMESTAMPTZ,
      error         TEXT
    );
    """)
    # Add job_id and visible columns to clusters table if not exist
    cur.execute("""
    ALTER TABLE clusters
    ADD COLUMN IF NOT EXISTS job_id BIGINT REFERENCES cluster_jobs(job_id);
    """)
    cur.execute("""
    ALTER TABLE clusters
    ADD COLUMN IF NOT EXISTS visible BOOLEAN NOT NULL DEFAULT FALSE;
    """)
    conn.commit()
    cur.close()
    conn.close()

create_cluster_jobs_and_clusters_tables()